In [3]:
import cloudknot as ck
ck.set_region('us-west-2')

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [1]:
def afq_cam_can(seed):
    import fury
    import xvfbwrapper
    import AFQ.data as afqd
    import AFQ.api as api
    import os.path as op
    import s3fs

    local_bids_folder = "./cam_can"
    bucket = "cam-can.afq-insight"
    s3_prefix = "raw/cc700/mri/pipeline/release004/BIDS_NRDG/BIDS_NRDG"

    study = afqd.S3BIDSStudy(
        "Cam-CAN",
        bucket,
        s3_prefix,
        subjects=1,
        anon=False,
        random_seed=seed)
    study.download(
        local_bids_folder,
        include_derivs="qsiprep-ck")
    
    myafq = api.AFQ(
        local_bids_folder,
        dmriprep="qsiprep-ck",
        segmentation="qsiprep-ck",
        seg_suffix='brain_mask',
        virtual_frame_buffer=True)
    myafq.export_all()

    s3fs.S3FileSystem().put(
        op.join(local_bids_folder, "afq"),
        op.join(bucket, s3_prefix, "afq"),
        recursive=True)
    

In [ ]:
afq_cam_can(1)

/home/john/miniconda3/envs/afq/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/john/miniconda3/envs/afq/lib/python3.7/site-packages/dipy/stats/__init__.py:7: UserWarning: The `dipy.stats` module is still under heavy development and functionality, as well as the API is likely to change in future versions of the software
  warnings.warn(w_string)
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
/home/john/miniconda3/envs/afq/lib/python3.7/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


Retrieving subject S3 keys
[                                        ] | 0% Completed |  0.0s

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


[                                        ] | 0% Completed |  0.5s

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


[                                        ] | 0% Completed |  1.0s

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


[                                        ] | 0% Completed |  1.6s

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


[########################################] | 100% Completed |  2.6s


In [6]:
afq_knot = ck.Knot(name='afq_cam_can-64gb-20811-0',
                  func=afq_cam_can,
                  base_image='libglxvfb:1',
                  image_github_installs="https://github.com/36000/pyAFQ.git@bds-fetch-fix",
                  pars_policies=('AmazonS3FullAccess',),
                  resource_type="SPOT",
                  bid_percentage=100,
                  memory=64000)

In [7]:
inputs = [1, 2, 3]

In [8]:
ft = afq_knot.map(inputs)

In [10]:
afq_knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
65d05a10-8020-405c-8da0-f03eb5c2788f        afq-cam-can-64gb-20811-0-0        FAILED   
